In [1]:
# State

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [6]:
# Case 1:
vec = np.array([1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)], dtype=np.float64)
true_object = State(c_sys, vec, on_para_eq_constraint=False)
# true_object = get_z0_1q(c_sys)
# true_object.vec

# Case 2:
# vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
# true_object = State(c_sys, vec)

# Case 3:
# vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
# true_object = State(c_sys, vec)

In [7]:
num_data = [100, 1000, 10000]
#n_rep = 100
n_rep = 10

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood Estimator(False)",
    "Least Squares Estimator(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, on_algo_eq_constraint=True, on_algo_ineq_constraint=True, eps_proj_physical=1e-13, seed=seed),
]
para_list = [
    True,
    False,
    True,
    False,
    False,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(4),
    WeightedProbabilityBasedSquaredError(4),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       loss=loss,
       loss_option=loss_option,
       algo=algo,
       algo_option=algo_option,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 20%|██        | 2/10 [00:00<00:00, 14.49it/s]Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator
 20%|██        | 2/10 [00:00<00:00, 15.38it/s]elapsed_time:0.010750305652618409[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator
 20%|██        | 2/10 [00:00<00:00, 10.81it/s]elapsed_time:0.010449830691019695[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator
 20%|██        | 2/10 [00:00<00:00, 11.97it/s]elapsed_time:0.014650082588195801[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator
  0%|          | 0/10 [00:00<?, ?it/s]elapsed_time:0.01410055955251058[min]

Case 4: Maximum-Likelihood Estimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LossMinimizatio

In [8]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,  # 「EstimationResultのリスト」のリスト
    case_name_list,  # ケース名のリスト。ここに指定した文字列が凡例に表示される
    true_object
)

In [9]:
report.export_report("sample_qst_report.pdf",
    estimation_results_list,   # 「EstimationResultのリスト」のリスト
    case_name_list,  # ケース名（この名前がMSEなどの凡例に表示される）
    estimator_list,  # 推定量のリスト
    true_object,  # True Object
    tester_objects,   # Tester Objectのリスト.
    loss_list=loss_list,
    loss_option_list=loss_option_list,
    algo_list=algo_list,
    algo_option_list=algo_option_list,
    seed=seed,  # 推定で使ったseed（オプション）
    computation_time=sum(elapsed_times) # 処理時間の合計（オプション）
)

100%|██████████| 10/10 [00:00<00:00, 10003.11it/s]​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)

In [10]:
report.export_report?

Signature:
report.export_report(
    path:str,
    estimation_results_list:List[List[_ForwardRef('EstimationResult')]],
    case_name_list:List[str],
    estimator_list:List[_ForwardRef('Estimator')],
    true_object:'QOperation',
    tester_objects:List[_ForwardRef('QOperation')],
    seed:Union[int, NoneType]=None,
    computation_time:Union[float, NoneType]=None,
    keep_tmp_files:bool=False,
    show_physicality_violation_check:bool=True,
)
Docstring: <no docstring>
File:      c:\users\satoyuki\myspace\github\tknrsgym\quara\quara\data_analysis\report.py
Type:      function
